# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1640919850>
├── 'a' --> tensor([[ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292]])
└── 'x' --> <FastTreeValue 0x7f1640919c40>
    └── 'c' --> tensor([[ 1.4042,  0.1280,  0.2811,  0.0819],
                        [-0.1038,  0.2039,  0.5301, -0.3892],
                        [ 0.9809,  0.6936, -0.5879,  0.5020]])

In [4]:
t.a

tensor([[ 1.2352,  0.1124,  0.4499],
        [ 0.7382,  1.2576, -0.7292]])

In [5]:
%timeit t.a

61.3 ns ± 0.0533 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1640919850>
├── 'a' --> tensor([[ 0.2807,  0.9217, -1.2276],
│                   [ 0.1510, -0.5546,  3.2212]])
└── 'x' --> <FastTreeValue 0x7f1640919c40>
    └── 'c' --> tensor([[ 1.4042,  0.1280,  0.2811,  0.0819],
                        [-0.1038,  0.2039,  0.5301, -0.3892],
                        [ 0.9809,  0.6936, -0.5879,  0.5020]])

In [7]:
%timeit t.a = new_value

57.6 ns ± 0.0384 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292]]),
    x: Batch(
           c: tensor([[ 1.4042,  0.1280,  0.2811,  0.0819],
                      [-0.1038,  0.2039,  0.5301, -0.3892],
                      [ 0.9809,  0.6936, -0.5879,  0.5020]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2352,  0.1124,  0.4499],
        [ 0.7382,  1.2576, -0.7292]])

In [11]:
%timeit b.a

52.5 ns ± 0.0352 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5227, -0.0537, -0.7453],
               [-1.6402,  0.2646, -1.8602]]),
    x: Batch(
           c: tensor([[ 1.4042,  0.1280,  0.2811,  0.0819],
                      [-0.1038,  0.2039,  0.5301, -0.3892],
                      [ 0.9809,  0.6936, -0.5879,  0.5020]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

849 ns ± 1.67 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 2.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1589c57f70>
├── 'a' --> tensor([[[ 1.2352,  0.1124,  0.4499],
│                    [ 0.7382,  1.2576, -0.7292]],
│           
│                   [[ 1.2352,  0.1124,  0.4499],
│                    [ 0.7382,  1.2576, -0.7292]],
│           
│                   [[ 1.2352,  0.1124,  0.4499],
│                    [ 0.7382,  1.2576, -0.7292]],
│           
│                   [[ 1.2352,  0.1124,  0.4499],
│                    [ 0.7382,  1.2576, -0.7292]],
│           
│                   [[ 1.2352,  0.1124,  0.4499],
│                    [ 0.7382,  1.2576, -0.7292]],
│           
│                   [[ 1.2352,  0.1124,  0.4499],
│                    [ 0.7382,  1.2576, -0.7292]],
│           
│                   [[ 1.2352,  0.1124,  0.4499],
│                    [ 0.7382,  1.2576, -0.7292]],
│           
│                   [[ 1.2352,  0.1124,  0.4499],
│                    [ 0.7382,  1.2576, -0.7292]]])
└── 'x' --> <FastTreeValue 0x7f1589c576d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 80.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1589c579a0>
├── 'a' --> tensor([[ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292],
│                   [ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292],
│                   [ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292],
│                   [ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292],
│                   [ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292],
│                   [ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292],
│                   [ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292],
│                   [ 1.2352,  0.1124,  0.4499],
│                   [ 0.7382,  1.2576, -0.7292]])
└── 'x' --> <FastTreeValue 0x7f1589c57a00>
    └── 'c' --> tensor([[ 1.4042,  0.1280,  0.2811,  0.0819],
                        [-0.1038,  0.2039,  0.5301, -0.3892],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 48.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.9 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2352,  0.1124,  0.4499],
                [ 0.7382,  1.2576, -0.7292]],
       
               [[ 1.2352,  0.1124,  0.4499],
                [ 0.7382,  1.2576, -0.7292]],
       
               [[ 1.2352,  0.1124,  0.4499],
                [ 0.7382,  1.2576, -0.7292]],
       
               [[ 1.2352,  0.1124,  0.4499],
                [ 0.7382,  1.2576, -0.7292]],
       
               [[ 1.2352,  0.1124,  0.4499],
                [ 0.7382,  1.2576, -0.7292]],
       
               [[ 1.2352,  0.1124,  0.4499],
                [ 0.7382,  1.2576, -0.7292]],
       
               [[ 1.2352,  0.1124,  0.4499],
                [ 0.7382,  1.2576, -0.7292]],
       
               [[ 1.2352,  0.1124,  0.4499],
                [ 0.7382,  1.2576, -0.7292]]]),
    x: Batch(
           c: tensor([[[ 1.4042,  0.1280,  0.2811,  0.0819],
                       [-0.1038,  0.2039,  0.5301, -0.3892],
                       [ 0.9809,  0.6936, -0.5879,  0.5020]],
         

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 364 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292],
               [ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292],
               [ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292],
               [ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292],
               [ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292],
               [ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292],
               [ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292],
               [ 1.2352,  0.1124,  0.4499],
               [ 0.7382,  1.2576, -0.7292]]),
    x: Batch(
           c: tensor([[ 1.4042,  0.1280,  0.2811,  0.0819],
                      [-0.1038,  0.2039,  0.5301, -0.3892],
                      [ 0.9809,  0.6936, -0.5879,  0.5020],
                      [ 1.4042,  0.1280,  0.2811,  0.0819],
                      [-0.1038,  

In [28]:
%timeit Batch.cat(batches)

141 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 2.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
